In [28]:
from collections import OrderedDict
import pickle

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from function.layer import Relu, SoftmaxWithLoss, Sigmoid, Affine
from function.etc import numerical_gradient

# 지뢰찾기 패턴 신경망

9개의 타일을 가진 샘플을 입력층에 전달한다.
샘플의 예시는 다음과 같다.
```
000
121
xxx
[0, 0, 0, 1, 2, 1, -1, -1, -1]
```
`x`의 경우는 아직 확인되지 않은 구간으로 입력층에 전달할 때는 -1의 값으로 전달.

이때의 경우 이를 만족하는 '올바른 지뢰찾기' 결과는 다음과 같다.
```
000
121
*2*
[0, 0, 0, 0, 0, 0, 1, 0, 1]
```
`*`는 지뢰이다.

이런 방식으로 여러 학습데이터를 만들어 학습시키는 것으로 해본다.

In [67]:
# 학습 데이터, 테스트 데이터 만들기
def neighbors(pos):
    l_min = np.max((pos[0] - 1, 0))
    l_max = np.min((pos[0] + 1, 4))
    r_min = np.max((pos[1] - 1, 0))
    r_max = np.min((pos[1] + 1, 4))
    ret = []
    for j in range(r_min, r_max + 1):
        for i in range(l_min, l_max + 1):
            if (i, j) != pos:
                ret.append((i, j))
    return np.array(ret)

def open_route(board_org, board_b, pos):
    if board_b[pos[0], pos[1]] != -1:
        return
    t = board_org[pos[0], pos[1]]
    board_b[pos[0], pos[1]] = t
    if t == 0:
        x = neighbors(pos)
        for i in x:
            open_route(board_org, board_b, tuple(i))

def closing(board, pos):
    l_min = np.max((pos[0] - 1, 0))
    l_max = np.min((pos[0] + 1, 4))
    r_min = np.max((pos[1] - 1, 0))
    r_max = np.min((pos[1] + 1, 4))
    return np.count_nonzero(board[l_min: l_max+1, r_min: r_max+1] == -1)


board_origin = np.zeros((5, 5), dtype=int)
board_hot = np.zeros_like(board_origin)
board_blind = np.full_like(board_origin, -1)

xx, yy = np.meshgrid(range(5), range(5))
A = np.array((xx.ravel(), yy.ravel())).T

t = A[np.random.choice(A.shape[0], 3, replace=False)]

board_origin[tuple(t[:2])] = -1
board_hot[tuple(t[:2])] = 1


for i in range(5):
    for j in range(5):
        if board_origin[i, j] == -1:
            continue
        board_origin[i, j] = closing(board_origin, (i, j))

open_pos = t[-1]# t[-1]

open_route(board_origin, board_blind, tuple(open_pos))

for row in board_origin:
    for c in row:
        if c == -1:
            print("*", end='')
        else:
            print(str(c), end='')
    print()
print("=====")
for row in board_blind:
    for c in row:
        if c == -1:
            print("x", end='')
        else:
            print(str(c), end= '')
    print()

B_origins = np.zeros((100, 5, 5), dtype=int)
B_hots = np.zeros((100, 5, 5), dtype=int)
B_blinds = np.zeros((100, 5, 5), dtype=int)
for k in range(100):
    board_origin = np.zeros((5, 5), dtype=int)
    board_hot = np.zeros_like(board_origin)
    board_blind = np.full_like(board_origin, -1)

    xx, yy = np.meshgrid(range(5), range(5))
    A = np.array((xx.ravel(), yy.ravel())).T

    t = A[np.random.choice(A.shape[0], 3, replace=False)]

    board_origin[tuple(t[:2])] = -1
    board_hot[tuple(t[:2])] = 1


    for i in range(5):
        for j in range(5):
            if board_origin[i, j] == -1:
                continue
            board_origin[i, j] = closing(board_origin, (i, j))

    open_pos = t[-1]# t[-1]

    open_route(board_origin, board_blind, tuple(open_pos))

    B_origins[k] = board_origin.copy()
    B_hots[k] = board_hot.copy()
    B_blinds[k] = board_blind.copy()

with open("dataset/minesweep_dataset/mine_board_data.pkl", 'wb') as f:
    data = {
        "B_origins": B_origins,
        "B_hots": B_hots,
        "B_blinds": B_blinds
    }
    pickle.dump(data, f)

0001*
00011
00000
01110
01*10
=====
0001x
00011
00000
01110
01x10


In [68]:
with open("dataset/minesweep_dataset/mine_board_data.pkl", 'rb') as f:
    data = pickle.load(f)
print(data["B_origins"][0])

[[ 0  0  1  1  1]
 [ 1  1  2 -1  1]
 [ 1 -1  2  1  1]
 [ 1  1  1  0  0]
 [ 0  0  0  0  0]]


In [ ]:
class MineSweepLayer:
    def __init__(self, io_size=25, hidden_size=50, weight_init_std=0.01):
        self.params = {
            "W1": weight_init_std * np.random.randn(io_size, hidden_size),
            "b1": weight_init_std * np.random.randn(hidden_size),
            "W2": weight_init_std * np.random.randn(hidden_size, io_size),
            "b2": weight_init_std * np.random.randn(io_size)
        }

        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])
        self.layers["Relu2"] = Relu()

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads
